# PREDECIR CLIMA

# Prueba 1

In [24]:
import requests
from datetime import datetime, timedelta

API_KEY = "H8M4YDHTPDD73C6JESETFVZFY"
ubicacion = "Gustavo A. Madero, Mexico City, MX"

# Años que quieres descargar
años = [2024, 2025]

for año in años:
    for mes in range(9, 13):
        fecha_inicio = f"{año}-{mes:02d}-01"
        # calcular último día del mes
        if mes == 12:
            fecha_fin = f"{año}-12-31"
        else:
            fecha_fin = (datetime(año, mes + 1, 1) - timedelta(days=1)).strftime("%Y-%m-%d")
        
        print(f"📅 Descargando {año}-{mes:02d} ({fecha_inicio} → {fecha_fin})...")

        url = (
            f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
            f"{ubicacion}/{fecha_inicio}/{fecha_fin}"
            f"?unitGroup=metric&include=days&key={API_KEY}&contentType=csv"
        )

        respuesta = requests.get(url)

        if respuesta.status_code == 200:
            nombre_archivo = f"clima_cdmx_{año}_{mes:02d}.csv"
            with open(nombre_archivo, "wb") as f:
                f.write(respuesta.content)
            print(f"✅ Guardado {nombre_archivo}")
        else:
            print(f"❌ Error {respuesta.status_code} en {año}-{mes:02d}")


📅 Descargando 2024-09 (2024-09-01 → 2024-09-30)...
❌ Error 429 en 2024-09
📅 Descargando 2024-10 (2024-10-01 → 2024-10-31)...
❌ Error 429 en 2024-10
📅 Descargando 2024-11 (2024-11-01 → 2024-11-30)...
❌ Error 429 en 2024-11
📅 Descargando 2024-12 (2024-12-01 → 2024-12-31)...
❌ Error 429 en 2024-12
📅 Descargando 2025-09 (2025-09-01 → 2025-09-30)...
❌ Error 429 en 2025-09
📅 Descargando 2025-10 (2025-10-01 → 2025-10-31)...
❌ Error 429 en 2025-10
📅 Descargando 2025-11 (2025-11-01 → 2025-11-30)...
❌ Error 429 en 2025-11
📅 Descargando 2025-12 (2025-12-01 → 2025-12-31)...
❌ Error 429 en 2025-12


In [ ]:
import pandas as pd
from glob import glob

# --- CARGAR Y UNIR TODOS LOS AÑOS ---
archivos = glob("clima_cdmx_*.csv")  # busca todos los CSV que empiecen así
df_list = []

for archivo in archivos:
    data = pd.read_csv(archivo)
    df_list.append(data)

# Unir todos los dataframes
df = pd.concat(df_list, ignore_index=True)
print("✅ Datos combinados:", df.shape)

NameError: name 'lat' is not defined

# Prueba 2

## Extracción de datos

In [301]:
# =========================================================
# 📊 Descarga y formato de datos climáticos Gustavo A. Madero (CDMX)
# Periodo: 2025-11-05T23:00 → 2025-11-09T16:00
# Formato EXACTO igual al histórico original
# =========================================================

import pandas as pd
import requests

# Coordenadas de Gustavo A. Madero, CDMX
latitude = 19.5047
longitude = -99.1469

# Fechas de interés
start_date = "2025-11-05"
end_date   = "2025-11-09"

# --- URL de Open-Meteo (API de histórico) ---
url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={latitude}&longitude={longitude}"
    f"&start_date={start_date}&end_date={end_date}"
    f"&hourly=temperature_2m,relative_humidity_2m,dewpoint_2m,pressure_msl,"
    f"precipitation,wind_speed_10m,wind_gusts_10m,wind_direction_10m,weathercode"
    f"&timezone=America/Mexico_City"
)

# --- Descarga de datos ---
response = requests.get(url)
data = response.json()

# --- Conversión a DataFrame ---
df = pd.DataFrame(data["hourly"])
df["time"] = pd.to_datetime(df["time"])

# --- Filtrar solo el rango exacto ---
mask = (df["time"] >= "2025-11-05T23:00") & (df["time"] <= "2025-11-09T19:00")
df = df.loc[mask].reset_index(drop=True)

# --- Conversión de unidades ---
# Viento de m/s → km/h
df["wind_speed_10m"] = df["wind_speed_10m"] * 3.6
df["wind_gusts_10m"] = df["wind_gusts_10m"] * 3.6

# --- Renombrar columnas para coincidir con tu formato ---
df_final = df.rename(columns={
    "time": "fecha_hora",
    "temperature_2m": "temperatura_C",
    "relative_humidity_2m": "humedad_%",
    "dewpoint_2m": "punto_rocio_C",
    "pressure_msl": "presion_hPa",
    "precipitation": "precipitacion_mm",
    "wind_speed_10m": "viento_velocidad_kmh",
    "wind_gusts_10m": "viento_rafaga_kmh",
    "wind_direction_10m": "viento_direccion_°",
    "weathercode": "codigo_clima_wmo"
})

# --- Reordenar columnas ---
df_final = df_final[[
    "fecha_hora", "temperatura_C", "humedad_%", "punto_rocio_C",
    "presion_hPa", "precipitacion_mm", "viento_velocidad_kmh",
    "viento_rafaga_kmh", "viento_direccion_°", "codigo_clima_wmo"
]]

# --- Formatear la fecha como 'YYYY-MM-DDTHH:MM' ---
df_final["fecha_hora"] = df_final["fecha_hora"].dt.strftime("%Y-%m-%dT%H:%M")

# --- Mostrar vista previa ---
print("=== DATOS CLIMÁTICOS DESCARGADOS ===")
display(df_final.head())

# --- Guardar en CSV con formato idéntico ---
nombre_archivo = "clima_GAM_2025-11-05_a_2025-11-09_formato_igual.csv"
df_final.to_csv(nombre_archivo, index=False)

print(f"\n✅ Archivo guardado con formato original: {nombre_archivo}")


=== DATOS CLIMÁTICOS DESCARGADOS ===


,fecha_hora,temperatura_C,humedad_%,punto_rocio_C,presion_hPa,precipitacion_mm,viento_velocidad_kmh,viento_rafaga_kmh,viento_direccion_°,codigo_clima_wmo
0,2025-11-05T23:00,14.3,66,8.0,1023.6,0.0,16.92,58.32,332,0
1,2025-11-06T00:00,13.2,66,7.0,1023.7,0.0,12.24,41.40,6,0
2,2025-11-06T01:00,12.7,68,6.8,1023.2,0.0,15.48,36.36,336,0
3,2025-11-06T02:00,12.1,70,6.8,1022.6,0.0,10.44,32.40,7,0
4,2025-11-06T03:00,11.2,75,7.0,1022.4,0.0,13.32,27.36,35,0



✅ Archivo guardado con formato original: clima_GAM_2025-11-05_a_2025-11-09_formato_igual.csv


## Predicción

In [346]:
# ================================================
# 1. Importar librerías
# ================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# ================================================
# 2. Cargar el dataset histórico
# ================================================
df = pd.read_csv("historico_clima_2022-2025_CDMX.csv")

print("Columnas:", df.columns.tolist())

# Asegurar que 'fecha_hora' es tipo datetime
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'], format='%Y-%m-%dT%H:%M', errors='coerce')

# Verificar la última hora real
print("Última hora en dataset:", df['fecha_hora'].max())


Columnas: ['fecha_hora', 'temperatura_C', 'humedad_%', 'punto_rocio_C', 'presion_hPa', 'precipitacion_mm', 'viento_velocidad_kmh', 'viento_rafaga_kmh', 'viento_direccion_°', 'codigo_clima_wmo']
Última hora en dataset: 2025-11-09 20:00:00


In [347]:
# Asegurar que la fecha sea tipo datetime y ordenar
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])
df = df.sort_values('fecha_hora')

# Agregar variables de horas anteriores (lags)
df["temp_hora_anterior"] = df["temperatura_C"].shift(1)
df["humedad_anterior"] = df["humedad_%"].shift(1)
df["viento_velocidad_anterior"] = df["viento_velocidad_kmh"].shift(1)
df = df.dropna()  # elimina primeras filas sin valores

# Variable objetivo: temperatura a la siguiente hora
df['temperatura_siguiente'] = df['temperatura_C'].shift(-1)

# Eliminar filas sin valor siguiente
df = df.dropna(subset=['temperatura_siguiente'])


In [348]:
df["hora"] = df["fecha_hora"].dt.hour
df["dia"] = df["fecha_hora"].dt.day
df["mes"] = df["fecha_hora"].dt.month
df["dia_semana"] = df["fecha_hora"].dt.dayofweek 

# Agregar más lags
for lag in [1, 2, 3, 6, 12, 24]:  # horas anteriores
    df[f"temp_lag_{lag}h"] = df["temperatura_C"].shift(lag)
    df[f"humedad_lag_{lag}h"] = df["humedad_%"].shift(lag)

# Features cíclicas para hora y mes
df["hora_sin"] = np.sin(2 * np.pi * df["hora"]/24)
df["hora_cos"] = np.cos(2 * np.pi * df["hora"]/24)
df["mes_sin"] = np.sin(2 * np.pi * df["mes"]/12)
df["mes_cos"] = np.cos(2 * np.pi * df["mes"]/12)

# Actualizar features
features = [
    'temperatura_C', 'humedad_%', 'punto_rocio_C', 'presion_hPa',
    'precipitacion_mm', 'viento_velocidad_kmh', 'viento_rafaga_kmh',
    'viento_direccion_°', 'hora_sin', 'hora_cos', 'mes_sin', 'mes_cos',
    'dia_semana', 'temp_hora_anterior', 'humedad_anterior', 
    'viento_velocidad_anterior', 'temp_lag_2h', 'temp_lag_3h'
]
target = 'temperatura_siguiente'

X = df[features]
y = df[target]


In [349]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

modelo = RandomForestRegressor(
    n_estimators=500,       # más árboles
    max_depth=15,           # evita sobreajuste
    min_samples_split=5,    # suaviza
    random_state=42,
    n_jobs=-1
)

modelo.fit(X_train, y_train)

# Evaluación
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"🔹 Error absoluto medio: {mae:.2f} °C")
print(f"🔹 R² (bondad de ajuste): {r2:.3f}")


🔹 Error absoluto medio: 0.48 °C
🔹 R² (bondad de ajuste): 0.976


In [350]:
# Asegurar formato datetime
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])

# Tomar la última hora disponible
hora_actual = df['fecha_hora'].max()

# Buscar esa fila
fila_actual = df.loc[df['fecha_hora'] == hora_actual, features]

if not fila_actual.empty:
    prediccion_temp = modelo.predict(fila_actual)[0]
    hora_prediccion = hora_actual + pd.Timedelta(hours=1)
    print(f"Predicción de temperatura para {hora_prediccion.strftime('%Y-%m-%dT%H:%M')}: {prediccion_temp:.2f} °C")
else:
    print("⚠️ No hay datos exactos para esa hora en el dataset.")


Predicción de temperatura para 2025-11-09T20:00: 14.90 °C


In [345]:
print("=== VERIFICACIÓN DE FECHAS ===")
print(f"Total de filas: {len(df)}")
print(f"Últimas 5 filas:")
print(df[['fecha_hora', 'temperatura_C']].tail())

print(f"\n¿Hay fechas que no se convirtieron?")
fechas_nulas = df['fecha_hora'].isna().sum()
print(f"Fechas nulas: {fechas_nulas}")

if fechas_nulas > 0:
    print("Filas con fechas problemáticas:")
    print(df[df['fecha_hora'].isna()].head())

=== VERIFICACIÓN DE FECHAS ===
Total de filas: 33810
Últimas 5 filas:
               fecha_hora  temperatura_C
33806 2025-11-09 14:00:00           23.7
33807 2025-11-09 15:00:00           24.1
33808 2025-11-09 16:00:00           23.0
33809 2025-11-09 17:00:00           21.1
33810 2025-11-09 18:00:00           18.7

¿Hay fechas que no se convirtieron?
Fechas nulas: 0
